# Adding Double Group Data

In [1]:
!rm ./data/CPGs.pkl

rm: ./data/CPGs.pkl: No such file or directory


In [2]:
%load_ext line_profiler
%load_ext autoreload
# provide cell timings
%load_ext autotime
# %autoreload 2
import sympy as sp
import numpy as np
%config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from qdef import *
from misc import *
from IPython.display import display, Math, Latex
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
import time

/Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/CPGs.pkl not found, regenerating ...
Parsing C_{1} ...
Parsing C_{i} ...
Parsing C_{2} ...
Parsing C_{s} ...
Parsing C_{2h} ...
Parsing D_{2} ...
Parsing C_{2v} ...
Parsing D_{2h} ...
Parsing C_{4} ...
Parsing S_{4} ...
Parsing C_{4h} ...
Parsing D_{4} ...
Parsing C_{4v} ...
Parsing D_{2d} ...
Parsing D_{4h} ...
Parsing C_{3} ...
Parsing S_{6} ...
Parsing D_{3} ...
Parsing C_{3v} ...
Parsing D_{3d} ...
Parsing C_{6} ...
Parsing C_{3h} ...
Parsing C_{6h} ...
Parsing D_{6} ...
Parsing C_{6v} ...
Parsing D_{3h} ...
Parsing D_{6h} ...
Parsing T ...
Parsing T_{h} ...
Parsing O ...
Parsing T_{d} ...
Parsing O_{h} ...


In [3]:
CPGs.get_group_by_label('C_{3v}').irrep_labels

[A_1, A_2, E]

In [4]:
display(Math(CPGs.get_group_by_label('C_{3v}').double_product_table.pretty_parse()))

<IPython.core.display.Math object>

In [5]:
display(Math(CPGs.get_group_by_label('C_{3v}').product_table.pretty_parse()))

<IPython.core.display.Math object>

In [4]:
!beep

zsh:1: command not found: beep


## Fixing the problem with the matrix inversion

In [ ]:
def rational_approx(x, N, min=0):
    '''
    Given  a number x this function returns a fraction
    that approximates it with a denominator that could
    be as large as N.
    '''
    if abs(x) <= min:
        return sp.S(0)
    if (int(x) == x):
        return sp.S(int(x))
    sign = 1
    if x < 0:
        sign = -1
        x = -x
    if x > 1:
        ix, dx = int(x), x - int(x)
    else:
        ix = 0
        dx = x
    exponent = -floor(log10(float(dx)))
    tens_multiplier = int(exponent-1)
    dx = dx*(10**tens_multiplier)
    divider = 1/(sp.S(10)**(sp.S(tens_multiplier)))
    sign = sign
    a, b = 0, 1
    c, d = 1, 1
    while (b <= N and d <= N):
        mediant = float(a+c)/(b+d)
        if dx == mediant:
            if b + d <= N:
                return sign*(sp.S(ix)+divider*sp.S(a+c)/sp.S(b+d))
            elif d > b:
                return sign*(sp.S(ix)+divider*sp.S(c)/sp.S(d))
            else:
                return sign*(sp.S(ix)+divider*sp.S(a)/sp.S(b))
        elif dx > mediant:
            a, b = a+c, b+d
        else:
            c, d = a+c,b+d
    if (b > N):
        return sign*(divider*sp.S(c)/sp.S(d) + sp.S(ix))
    else:
        return sign*(divider*sp.S(a)/sp.S(b) + sp.S(ix))

def rational_simplify(sympy_expr, N=10000):
    '''
    Given a sympy expression this function takes it and
    finds rational  approximations (perhaps including a
    square root).

    Example
    -------

    >> rational_simplify(2.31099*sp.Symbol('x') - 1.14)
    >>> 9 * sqrt(546) * x / 91 - sqrt(130)/10
    '''
    sympy_dict = sympy_expr.as_coefficients_dict()
    for k,v in sympy_dict.items():
        if isinstance(v, sp.core.numbers.Float):
            n = N
            simpler = rational_approx(v, n)
            # If the thing was approximated to zero
            # escalate the precision.
            while simpler == 0:
                n = 10*n
                simpler = rational_approx(v, n)
            sympy_dict[k] = simpler
    total = sum([k*v for k,v in sympy_dict.items()])
    return total

def square_rational_approx(x, N,min=0):
    '''
    Given a number x this algorithm finds the best  rational
    approximation to its square, and then returns the signed
    square root of that.
    '''
    if x < 0:
        sign = -1
        x = -x
    else:
        sign = 1
    y = x*x
    squared_approx = sign*sp.sqrt(rational_approx(y,N,min))
    normal_approx = sign*rational_approx(x,N*2,min)
    d1 = abs(x-squared_approx)
    d2 = abs(x-normal_approx)
    return squared_approx
    if d1 < d2:
        return squared_approx
    else:
        return normal_approx

def double_group_matrix_inverse(double_group_chartable, group_label):
    if group_label == 'O_{h}':
        N = 10000
    else:
        N = 1000
    chartable = double_group_chartable.T
    num_rows = chartable.rows
    num_cols = chartable.cols
    chararray = np.array(chartable).astype(np.complex128)
    chararrayinverse = np.linalg.inv(chararray)
    charinverse = sp.N(sp.Matrix(chararrayinverse),chop=True)
    simple_inverse = []
    num_rows = charinverse.rows
    num_cols = charinverse.cols
    for row_idx in range(num_rows):
        row = [sp.I*square_rational_approx(sp.im(charinverse[row_idx,col_idx]),N,1e-6) + square_rational_approx(sp.re(charinverse[row_idx,col_idx]),N,1e-6) for col_idx in range(num_cols)]
        simple_inverse.append(row)
    simple_inverse = sp.Matrix(simple_inverse)
    check = simple_inverse*chartable
    check = sp.re(check) + sp.I * sp.im(check)
    check = sp.N(check, chop=True)
    elements = (set(list(check)))
    unit_check = (len(elements) == min(2, num_rows) and (list(sorted(list(elements))) == [0,1]))
    if not unit_check:
        print("ERROR in matrix inversion")
        break
    return simple_inverse

In [ ]:
inverse_matrices = {}
for group_label in CPGs.all_group_labels:
    if group_label == 'O_{h}':
        N = 10000
    else:
        N = 1000
    print(group_label)
    chartable = CPGs.get_group_by_label(group_label).double_character_table.T
    num_rows = chartable.rows
    num_cols = chartable.cols
    chararray = np.array(chartable).astype(np.complex128)
    chararrayinverse = np.linalg.inv(chararray)
    charinverse = sp.N(sp.Matrix(chararrayinverse),chop=True)
    simple_inverse = []
    num_rows = charinverse.rows
    num_cols = charinverse.cols
    for row_idx in range(num_rows):
        row = [sp.I*square_rational_approx(sp.im(charinverse[row_idx,col_idx]),N,1e-6) + square_rational_approx(sp.re(charinverse[row_idx,col_idx]),N,1e-6) for col_idx in range(num_cols)]
        simple_inverse.append(row)
    simple_inverse = sp.Matrix(simple_inverse)
    check = simple_inverse*chartable
    check = sp.re(check) + sp.I * sp.im(check)
    check = sp.N(check, chop=True)
    elements = (set(list(check)))
    unit_check = (len(elements) == min(2, num_rows) and (list(sorted(list(elements))) == [0,1]))
    if not unit_check:
        print("ERROR in matrix inversion")
        break
    inverse_matrices[group_label] = simple_inverse
    